# Audio Delay using the Python Standard Library

_Delay_ is a fundamental audio effect. The idea of playing a sound and repeating it once after some time is simple, but it is used extensively in music production both on its own and as the basis of other effects like _reverb_, _chorus_, and _flanging_. Implementing this effect is a way of testing audio support in the standard library and understanding how delay works.

This notebook has been tested in Python 3.4.3. 

### Get some audio
The [Wave][] module handles input and output of [Wav][] audio files. Audio data is represented as a bytes object, which makes this standard library module less useful than some other modules that return a numpy array (e.g. [audiolab][]).

To work with the data in the bytes object we'll also need to know the number of [bytes per sample][] and the [sample frequency][]. These examples only support mono audio, so the nchannels parameter will need to be equal to 1.

[Wave]: https://docs.python.org/3.4/library/wave.html
[Wav]: https://en.wikipedia.org/wiki/WAV
[bytes per sample]: https://en.wikipedia.org/wiki/Audio_bit_depth
[audiolab]: https://pypi.python.org/pypi/scikits.audiolab
[sample frequency]: https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Audio_sampling

In [1]:
import wave

def input_wave(filename,frames=10000000): #10000000 is an arbitrary large number of frames
    with wave.open(filename,'rb') as wave_file:
        params=wave_file.getparams()
        audio=wave_file.readframes(frames)  
        if params.nchannels!=1:
            raise Exception("The input audio should be mono for these examples")
    return params, audio

#output to file so we can use ipython notebook's Audio widget
def output_wave(audio, params, stem, suffix):
    #dynamically format the filename by passing in data
    filename=stem.replace('.wav','_{}.wav'.format(suffix))
    with wave.open(filename,'wb') as wave_file:
        wave_file.setparams(params)
        wave_file.writeframes(audio)

This is a short mono audio clip for demonstration:

In [2]:
from IPython.display import Audio,display
display(
    Audio(filename='wavs/Trumpet.wav')
    )

In [ ]:
trumpet_params, trumpet_bytes = input_wave('wavs/Trumpet.wav') #must be mono
print("Bytes per Sample: {}".format(trumpet_params.sampwidth), 
      "Bytes per Sample: {}".format(trumpet_params.framerate),
      "First 10 bytes:", trumpet_bytes[:10], sep='\n')

### Implement a delay function
The simplest delay function will create a copy of the input, add some silence (0's in the bytes object) to the beginning of the copy, and combine it with the original input. The __add__ function from [Audioop](https://docs.python.org/3.4/library/audioop.html) will add the two bytes objects together. Audioop.add requires both pieces of audio to have the same length, so we also need to cut off the end of the copy. 


In [ ]:
from audioop import add

def delay(audio_bytes,params,offset_ms):
    #calculate the number of bytes which corresponds to the offset in milliseconds, 
    #depending on sampwith and framerate
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some empty space of offset-length
    beginning= b'\0'*offset
    #remove the same amount of space from the end
    end= audio_bytes[:-offset]
    return add(audio_bytes, beginning+end, params.sampwidth)

In [ ]:
#1-second delay
delayed_bytes_1000=delay(trumpet_bytes,trumpet_params,1000)
output_wave(delayed_bytes_1000, trumpet_params, 'wavs/Trumpet.wav','delay_{}'.format(1000))
#700 ms delay
delayed_bytes_700=delay(trumpet_bytes,trumpet_params,700)
output_wave(delayed_bytes_700, trumpet_params, 'wavs/Trumpet.wav','delay_{}'.format(700))

display(
    Audio('wavs/Trumpet_delay_1000.wav'),
    Audio('wavs/Trumpet_delay_700.wav')
    )

### Change the delayed audio's volume
To make this sound more like a realistic echo, we can change the volume of the delayed audio by multiplying it using audioop.mul. Note that multiplying each sample by one half is not the same as reducing the percieved loudness by one half.

In [ ]:
from audioop import mul
#new delay function with factor
def delay(audio_bytes,params,offset_ms,factor=1):
    #calculate the number of bytes which corresponds to the offset in milliseconds
    #depending on sampwith and framerate
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some empty space of offset-length
    beginning= b'\0'*offset
    #remove the same amount of space from the end
    end= audio_bytes[:-offset]
    #multiply the delayed portion by a factor
    multiplied_end= mul(audio_bytes[:-offset],params.sampwidth,factor)
    return add(audio_bytes, beginning+ multiplied_end, params.sampwidth)

### Test out offset lengths and volume factors

The best way to understand how different parameters affect the final sound is to try out a lot of examples. delay_to_file is a helper function to speed up this process.

In [ ]:
#helper function to try out lots of delays
def delay_to_file(audio_bytes,params,offset_ms,file_stem,factor=1):
    delayed_bytes=delay(audio_bytes,params,offset_ms,factor)
    output_wave(delayed_bytes, params, file_stem,'delay_{}_{}'.format(offset_ms,factor))

A single delay of 1-2 seconds is too long to sound like a natural echo, so this can be used for a musical effect.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=2000,file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1000,file_stem='wavs/Trumpet.wav',factor=.5)

from IPython.display import display
display(
    Audio('wavs/Trumpet_delay_2000_0.5.wav'),
    Audio('wavs/Trumpet_delay_1000_0.5.wav')
    )

Adding 250-400 ms of delay gives the impression of a natural echo that might occur in a larger physical space.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.25)

display(
    Audio('wavs/Trumpet_delay_400_0.5.wav'),
    Audio('wavs/Trumpet_delay_400_0.25.wav'),
    Audio('wavs/Trumpet_delay_250_0.5.wav'),
    Audio('wavs/Trumpet_delay_250_0.25.wav')
    )

As delays get shorter, around 125 ms or less, the effect starts to be percieved as a single sound rather than a distinct echo. This produces the effect of a [comb filter](https://en.wikipedia.org/wiki/Comb_filter) as certain frequencies get louder or softer due to interference. On this trumpet sample, 1-5 ms delay sounds a lot like a trumpet played through a [mute](http://www.summersong.net/teacher/trumpetlessons/brassinstrumentmutes/).

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=125,  file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=75,  file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=20,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=5,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1,  file_stem='wavs/Trumpet.wav',factor=.75)

display(
    Audio('wavs/Trumpet_delay_125_0.5.wav'),
    Audio('wavs/Trumpet_delay_75_0.5.wav'),
    Audio('wavs/Trumpet_delay_20_0.75.wav'),
    Audio('wavs/Trumpet_delay_5_0.75.wav'),
    Audio('wavs/Trumpet_delay_1_0.75.wav')
    )       

### Multiple delays

In [ ]:
from warnings import warn

def multi_delay(audio_bytes,params,offset_ms,factor=1,num=1):
    if factor>0.7 and num >3:
        warn("These settings may produce a very loud audio file. Please use caution when listening")
    #calculate the number of bytes which corresponds to the offset in milliseconds, 
    #depending on sampwith and framerate
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #create a copy of the original to apply the delays
    delayed_bytes=audio_bytes
    #at each step of the loop, "
    for i in range(num):
        end = delayed_bytes[:-offset]
        #multiplied_end=end
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        beginning = b'\0'*offset
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes

In [ ]:
def multi_delay_to_file(audio_bytes,params,offset_ms,file_stem,factor=1,num=1):
    echoed_bytes=multi_delay(audio_bytes,params,offset_ms,factor,num)
    output_wave(echoed_bytes, params, file_stem,'multi_delay_{}_{}_{}'.format(offset_ms,factor,num))

In [ ]:
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=50,file_stem='wavs/Trumpet.wav',factor=.76,num=10)
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=250,file_stem='wavs/Trumpet.wav',factor=.7,num=10)
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=1000,file_stem='wavs/Trumpet.wav',factor=.9,num=3)
display(
    Audio(filename='wavs/Trumpet_multi_delay_50_0.76_10.wav'),
    Audio(filename='wavs/Trumpet_multi_delay_250_0.7_10.wav'),
    Audio(filename='wavs/Trumpet_multi_delay_1000_0.9_3.wav'),
)

### Leave space at the end
The multi-delays above output a sound of the same length as the original. To give more time to hear the effects, let's increase the length to allow every delayed audio to finish.

In [ ]:
#TODO - the amount of space is not correct

# add extra space at the end for the delays
def multi_delay(audio_bytes,params,offset_ms,factor=1,num=1):
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    longer_bytes=audio_bytes+b'\0'*offset*(num+2) #leave enough room for the original plus the delays
    for i in range(num):
        end = longer_bytes[:-offset*(i+1)]
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        beginning = b'\0'*offset*(i+1)
        longer_bytes= add(longer_bytes, beginning+multiplied_end, params.sampwidth)
    return longer_bytes

In [ ]:
from IPython.display import FileLinks
FileLinks('wavs/')